In [1]:
import requests
import os
from dotenv import load_dotenv
import time
import pandas as pd
import import_ipynb

In [2]:
import open_weather_coordinates

Stadt: London, Land: GB, Breitengrad: 51.5073219, Längengrad: -0.1276474
Stadt: Paris, Land: FR, Breitengrad: 48.8588897, Längengrad: 2.3200410217200766
Stadt: Madrid, Land: ES, Breitengrad: 40.4167047, Längengrad: -3.7035825
Stadt: Frankfurt, Land: DE, Breitengrad: 50.1106444, Längengrad: 8.6820917
Stadt: Zurich, Land: CH, Breitengrad: 47.3744489, Längengrad: 8.5410422
Stadt: Moscow, Land: RU, Breitengrad: 55.7504461, Längengrad: 37.6174943
Stadt: Amsterdam, Land: NL, Breitengrad: 52.3727598, Längengrad: 4.8936041
Stadt: Lisbon, Land: PT, Breitengrad: 38.7077507, Längengrad: -9.1365919
Stadt: Rome, Land: IT, Breitengrad: 41.8933203, Längengrad: 12.4829321
Stadt: Dublin, Land: IE, Breitengrad: 53.3498006, Längengrad: -6.2602964
Stadt: Vienna, Land: AT, Breitengrad: 48.2083537, Längengrad: 16.3725042


In [3]:
coordinates = {}

for city in open_weather_coordinates.cities_coordinates:
    city_name = city["city_name"]
    coordinates[city_name] = {"lat":city["lat"], "lon": city["lon"]}

In [4]:
# Lade Umgebungsvariablen aus der .env-Datei (z. B. API-Schlüssel)
load_dotenv()

# API-Schlüssel aus der .env-Datei laden
api_key = os.getenv('API_KEY')

In [ ]:
# Berechne den Unix-Zeitstempel von vor 24 Stunden
#start = current_time - (7*86400)  # 24 Stunden = 86400 Sekunden

In [5]:
def fetch_weather(coordinates, api_key):
    # Aktuelles Datum in Unix-Timestamp umwandeln
    current_time = int(time.time())  # Aktueller Unix-Zeitstempel
    one_week_in_seconds = 7 * 86400
    all_data = []  # Liste zum Speichern der Daten in flacher Struktur

    for place, coord in coordinates.items():
        lat = coord['lat']
        lon = coord['lon']

        for week in range(52):
            # Berechne den Start- und Endzeitpunkt für jede Woche
            start = current_time - ((week + 1) * one_week_in_seconds)
            end = current_time - (week * one_week_in_seconds)
            
            # API-Aufruf für historische Wetterdaten
            api_url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=day&start={start}&end={end}&appid={api_key}"

            try:
                # API GET-Request senden
                response = requests.get(api_url)

                if response.status_code == 200:
                    data = response.json()
                    
                    # Iteriere durch jeden Dateneintrag und extrahiere relevante Felder
                    for entry in data.get('list', []):
                        record = {
                            "place": place,
                            "latitude": lat,
                            "longitude": lon,
                            "week": week + 1,
                            "timestamp": entry.get("dt"),
                            "date": time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(entry['dt'])),
                            "temperature": entry["main"]["temp"],
                            "humidity": entry["main"]["humidity"],
                            "pressure": entry["main"]["pressure"],
                            "wind_speed": entry["wind"]["speed"],
                            "weather_main": entry["weather"][0]["main"],
                            "weather_description": entry["weather"][0]["description"]
                        }
                        # Füge den einzelnen Datensatz der flachen Datenstruktur hinzu
                        all_data.append(record)
                else:
                    print(f"Fehlerhafte Anfrage für {place} ({lat}, {lon}). Statuscode: {response.status_code}")

            except requests.exceptions.RequestException as e:
                print(f"Ein Fehler ist aufgetreten für {place} ({lat}, {lon}): {e}")

    return all_data  # Gib die gesammelten Daten in flacher Struktur zurück


In [ ]:
#funktion für 6-9 uhr 
def fetch_weather_morning(coordinates, api_key):
    current_time = int(time.time())  # Aktueller Unix-Zeitstempel
    one_week_in_seconds = 7 * 86400
    all_data = []  # Liste zum Speichern der Daten in flacher Struktur

    for place, coord in coordinates.items():
        lat = coord['lat']
        lon = coord['lon']

        for week in range(52):  # Für jede Woche
            start = current_time - ((week + 1) * one_week_in_seconds)
            end = current_time - (week * one_week_in_seconds)
        
            # API-Aufruf für historische Wetterdaten
            api_url = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=day&start={start}&end={end}&appid={api_key}"

            try:
                # API GET-Request senden
                response = requests.get(api_url)

                if response.status_code == 200:
                    data = response.json()

                    # Filtern der Daten für den Zeitraum 6:00 Uhr bis 9:00 Uhr
                    for entry in data.get("list", []):
                        entry_hour = int(time.strftime('%H', time.gmtime(entry['dt'])))
                        if 6 <= entry_hour <= 9:  # Nur Einträge von 6:00 bis 9:00 Uhr speichern
                            observation = {
                                "place": place,
                                "latitude": lat,
                                "longitude": lon,
                                "week": week + 1,
                                "timestamp": entry.get("dt"),
                                "date": time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(entry['dt'])),
                                "temperature": entry["main"]["temp"],
                                "humidity": entry["main"]["humidity"],
                                "pressure": entry["main"]["pressure"],
                                "wind_speed": entry["wind"]["speed"],
                                "weather_main": entry["weather"][0]["main"],
                                "weather_description": entry["weather"][0]["description"]
                            }
                            all_data.append(observation)
                else:
                    print(f"Fehlerhafte Anfrage für {place} ({lat}, {lon}). Statuscode: {response.status_code}")

            except requests.exceptions.RequestException as e:
                print(f"Ein Fehler ist aufgetreten für {place} ({lat}, {lon}): {e}")

    return all_data  # Gib die gesammelten Daten zurück


In [10]:
data_morning = fetch_weather_morning(coordinates, api_key)

In [11]:
london_data = [entry for entry in data_morning if entry['place'] == 'London']
for entry in london_data:
    print(entry)

{'place': 'London', 'latitude': 51.5073219, 'longitude': -0.1276474, 'week': 1, 'timestamp': 1730538000, 'date': '2024-11-02 09:00:00', 'temperature': 285.24, 'humidity': 91, 'pressure': 1031, 'wind_speed': 2.57, 'weather_main': 'Clouds', 'weather_description': 'overcast clouds'}
{'place': 'London', 'latitude': 51.5073219, 'longitude': -0.1276474, 'week': 1, 'timestamp': 1730613600, 'date': '2024-11-03 06:00:00', 'temperature': 284.93, 'humidity': 84, 'pressure': 1029, 'wind_speed': 4.12, 'weather_main': 'Clouds', 'weather_description': 'overcast clouds'}
{'place': 'London', 'latitude': 51.5073219, 'longitude': -0.1276474, 'week': 1, 'timestamp': 1730617200, 'date': '2024-11-03 07:00:00', 'temperature': 284.88, 'humidity': 83, 'pressure': 1029, 'wind_speed': 3.6, 'weather_main': 'Clouds', 'weather_description': 'overcast clouds'}
{'place': 'London', 'latitude': 51.5073219, 'longitude': -0.1276474, 'week': 1, 'timestamp': 1730620800, 'date': '2024-11-03 08:00:00', 'temperature': 284.93,